In [ ]:
import cv2
import scipy
import matplotlib.pyplot as plt
import imutils
import numpy as np
from scipy.spatial import distance as dist
from imutils import perspective
from imutils import contours

In [ ]:
def midpoint(a, b):
    return tuple(((a + b)/2).astype(int))

def get_mm_per_pixel(real_object_size, cnts):
    
    '''
    Select smallest contour
    select random edge from it
    scale length of random edge with real_object_size
    '''
    
    cnt_areas = [cv2.contourArea(cnt) for cnt in cnts]
    cnt_areas.sort()
    smallest_area = cnt_areas[0]
    smallest_cnt = [cnt for cnt in cnts if cv2.contourArea(cnt) == smallest_area]

    rect = cv2.minAreaRect(smallest_cnt[0])
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    (a, b, c, d) = box
    length = dist.euclidean(a, b)
    return real_object_size/length    

def filter_biggest_contour_areas(cnts, num):
    '''
    get num biggest areas from cnts
    '''
    cnt_areas = [cv2.contourArea(cnt) for cnt in cnts]
    cnt_areas.sort()
    cnts = [cnt for cnt in cnts if cv2.contourArea(cnt) >= cnt_areas[-num]]
    return cnts

# Configuration

In [ ]:
num_objects_on_image = 2
binarization_lim = 120 #from which rgb value on is 1 or 0?

In [ ]:
raw_img = cv2.imread("IMG_20200504_153955.jpg")
grey = cv2.cvtColor(raw_img, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(grey, binarization_lim, 255, 0)
plt.figure(figsize=(20,10))
plt.imshow(thresh)

In [ ]:
blurred = cv2.GaussianBlur(thresh, (7, 7), 0)
edged = cv2.Canny(blurred, 50, 100)
edged = cv2.dilate(edged, None, iterations=1)
edged = cv2.erode(edged, None, iterations=1)
plt.figure(figsize=(20,10))
plt.imshow(edged)

In [ ]:
output = raw_img.copy()

(cnts, _) = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

cnts = filter_biggest_contour_areas(cnts, num_objects_on_image)
mm_per_pixel = get_mm_per_pixel(24, cnts)

print(mm_per_pixel)

for contour in cnts:
    rect = cv2.minAreaRect(contour)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    cv2.drawContours(output,[box],0,(0,0,255),4)
    
    (a, b, c, d) = box
    ab = dist.euclidean(a, b)
    bc = dist.euclidean(b, c)
    cd = dist.euclidean(c, d)
    da = dist.euclidean(d, a)

    cv2.putText(output, "{:.1f}mm".format(ab*mm_per_pixel),midpoint(a,b), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 2)
    cv2.putText(output, "{:.1f}mm".format(bc*mm_per_pixel),midpoint(b,c), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 2)
    cv2.putText(output, "{:.1f}mm".format(cd*mm_per_pixel),midpoint(c,d), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 2)
    cv2.putText(output, "{:.1f}mm".format(da*mm_per_pixel),midpoint(d,a), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 2)
    

plt.figure(figsize=(20,10))
plt.imshow(output)
plt.savefig('out.png')
    

# Playground

In [ ]:
lines = cv2.HoughLinesP(img, 1, np.pi/180, 10, maxLineGap = 500)
for line in lines:
    x1, y1, x2, y2 = line[0]
    img = cv2.line(edged, (x1,y1), (x2, y2), (0,255,0), 3)
plt.imshow(img)

In [ ]:
img = cv2.equalizeHist(img)
plt.imshow(img)

In [ ]:
ret, thresh = cv2.threshold(img, 120, 255, 0)
plt.imshow(thresh)